# Кто хочет стать миллиардером?


# Анализ датасета [Billionaires Statistics Dataset](https://www.kaggle.com/datasets/nelgiriyewithana/billionaires-statistics-dataset): факторы богатства и успеха

Мы хотим исследовать данные о миллиардерах мира с целью выявления закономерностей, связывающих их состояние с демографическими, географическими и экономическими факторами. Мы проанализируем, как отрасли бизнеса, страна проживания, возраст, образование и другие параметры влияют на финансовый успех. Результаты помогут понять:

- Какие страны и отрасли «производят» больше всего миллиардеров;  
- Существует ли гендерный разрыв в распределении богатства;  
- Как налоговая политика и экономические показатели связаны с благосостоянием сверхбогатых.  

###Гипотезы для проверки:

1. В среднем состояние IT-миллиардеров больше, чем состояние миллиардеров из других областей.  

2. Чем выше налоговая ставка в стране, тем меньше в этой стране миллиардеров.

3. Средний возраст self-made миллиардеров в IT ниже, чем средний возраст self-made миллиардеров в традиционных отраслях.

4. Большинство русских сверхбогатых людей заработали состояние в металлургической промышленности.

5. В странах с высоким ВВП миллиардеры в среднем богаче, чем в странах с низким ВВП.

6. В странах с высоким уровнем образования больше self-made миллиардеров.

###Модели

- Классификация отрасли с помощью Random Forest.  
- Регрессия для предсказания размера состояния на основе экономических показателей страны.  

###Методы

1. ANOVA и t-тест.

2. Расчет корреляции Пирсона и построение scatter plot с трендовой линией.      

###Карта богатства
Для углубленного изучения географического распределения богатства мы:  
1. Построим интерактивную карту, где размер маркера - состояние миллиардера, цвет маркера - отрасль.

2. Применим алгоритм DBScan для выделения кластеров с высокой концентрацией сверхбогатых людей:    
   - Цель: Обнаружить регионы, где сосредоточены миллиардеры  
   - Метрика: Оптимальные параметры eps и min_samples подбираются через GridSearch  

####Команда: Журавлев Артемий, Губенко Олеся







In [6]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('Billionaires Statistics Dataset.csv')
df.head(10)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,6.705989e+07,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
5,6,104000,Technology,Bill Gates,67.0,United States,Medina,Microsoft,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
6,7,94500,Media & Entertainment,Michael Bloomberg,81.0,United States,New York,Bloomberg LP,Media & Entertainment,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
7,8,93000,Telecom,Carlos Slim Helu & family,83.0,Mexico,Mexico City,Telecom,Telecom,Mexico,...,3.6,"$1,258,286,717,125",40.2,105.8,75.0,13.1,55.1,1.260140e+08,23.634501,-102.552784
8,9,83400,Diversified,Mukesh Ambani,65.0,India,Mumbai,Diversified,Diversified,India,...,7.7,"$2,611,000,000,000",28.1,113.0,69.4,11.2,49.7,1.366418e+09,20.593684,78.962880
9,10,80700,Technology,Steve Ballmer,67.0,United States,Hunts Point,Microsoft,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891


###Датасет содержит следующие признаки:

**rank**: Ранг миллиардера по уровню богатства (место в рейтинге).

**finalWorth**: Итоговое состояние миллиардера в долларах США (чистый капитал).

**category**: Категория или отрасль, в которой работает бизнес миллиардера.

**personName**: Имя миллиардера.

**age**: Возраст миллиардера.

**country**: Страна проживания миллиардера.

**city**: Город проживания миллиардера.

**source**: Источник богатства миллиардера.

**industries**: Отрасли, связанные с бизнес-интересами миллиардера.

**countryOfCitizenship**: Страна гражданства миллиардера.

**organization**: Название организации/компании, связанной с миллиардером.

**selfMade**: Указывает, является ли миллиардер self-made.

**gender**: Пол миллиардера.

**birthDate**: Дата рождения миллиардера.

**lastName**: Фамилия миллиардера.

**firstName**: Имя миллиардера.

**title**: Должность в компании.

**date**: Дата сбора данных.

**state**: Штат проживания (для США).

**residenceStateRegion**: Регион/штат проживания.

**birthYear**: Год рождения.

**birthMonth**: Месяц рождения.

**birthDay**: День рождения.

**cpi_country**: Индекс потребительских цен (ИПЦ) в стране миллиардера.

**cpi_change_country**: Изменение ИПЦ в стране миллиардера.

**gdp_country**: ВВП страны миллиардера.

**gross_tertiary_education_enrollment**: Доля населения с высшим образованием в стране.

**gross_primary_education_enrollment_country**: Доля населения с начальным образованием в стране.

**life_expectancy_country**: Ожидаемая продолжительность жизни в стране.

**tax_revenue_country_country**: Налоговые поступления в стране.

**total_tax_rate_country**: Общая налоговая ставка в стране.

**population_country**: Население страны.

**latitude_country**: Географическая широта страны.

**longitude_country**: Географическая долгота страны.

